In [ ]:
import smash
import numpy as np
import pandas as pd
from datetime import timedelta

In [ ]:
def _timestep_1year(st, dt, n_ts, by=None):

    if by is None:
        by = 1
    elif isinstance(by, str):
        if by=='hour':
            by = int(60*60/dt)
        elif by=='day':
            by = int(24*60*60/dt)
        elif by=='month':
            by = int(365/12*24*60*60/dt)

    timestep = np.arange(1, int(365*24*60*60/dt) + 1)

    defst = f"{pd.to_datetime(st).year}-08-01 00:00:00"

    if pd.Timestamp(st) < pd.Timestamp(defst):
        timestamps = pd.date_range(start=st, end=defst, freq=timedelta(seconds=dt))
        s_ind = timestep.size - (len(timestamps) - 1)
    else:
        timestamps = pd.date_range(start=defst, end=st, freq=timedelta(seconds=dt))
        s_ind = len(timestamps) - 1

    return np.array([timestep[(s_ind+i)%len(timestep)]//by for i in range(n_ts)])

In [ ]:
def model_to_df(model: smash.Model, 
                target_present: bool = True, 
                precip: bool = True, 
                pot_evapot: bool = True, 
                precip_ind: bool = True, 
                gauge: bool = None):

    dict_df = {}

    # % Catchment code for info
    code = model.mesh.code 
    code = np.tile(code, model.setup.ntime_step)
    dict_df["code"] = code

    # % Timestep for info
    timestep = np.repeat(np.arange(model.setup.ntime_step), model.mesh.ng)
    dict_df["timestep"] = timestep

    # % Meaningful timestep in year for learning
    tsy = _timestep_1year(model.setup.start_time, model.setup.dt, model.setup.ntime_step)
    tsy = np.repeat(tsy, model.mesh.ng)
    dict_df["timestep_in_year"] = tsy

    # day_in_year = _timestep_1year(model.setup.start_time, model.setup.dt, model.setup.ntime_step, by="day")
    # day_in_year = np.repeat(day_in_year, model.mesh.ng)
    # dict_df["day_in_year"] = day_in_year

    # month_in_year = _timestep_1year(model.setup.start_time, model.setup.dt, model.setup.ntime_step, by="month")
    # month_in_year = np.repeat(month_in_year, model.mesh.ng)
    # dict_df["month_in_year"] = month_in_year.astype(str)

    # % Simumated discharges
    qs = model.sim_response.q
    dict_df["discharge_sim"] = qs.flatten(order="F")

    # % Bias
    if target_present:
        qo = model.obs_response.q.copy()
        qo[qo<0] = np.nan
        bias = qo - qs
        dict_df["bias"] = bias.flatten(order="F")

    # % Mean Precipitation
    if precip:
        prcp = model.atmos_data.mean_prcp.copy()
        prcp[prcp<0] = np.nan
        dict_df["precipitation"] = prcp.flatten(order="F")

    # % PET
    if pot_evapot:
        pet = model.atmos_data.mean_pet.copy()
        pet[pet<0] = np.nan
        dict_df["pet"] = pet.flatten(order="F")

    # % Precipitation indices
    if precip_ind:
        prcp_ind = smash.precipitation_indices(model)
        d1 = prcp_ind.d1.copy()
        d1[np.isnan(d1)] = -1
        d2 = prcp_ind.d2.copy()
        d2[np.isnan(d2)] = -1
        dict_df["d1"] = d1.flatten(order="F")
        dict_df["d2"] = d2.flatten(order="F")

    df = pd.DataFrame(dict_df)

    if not gauge is None:
        df = df[df["code"].isin(gauge)]

    return df

In [ ]:
model_train = smash.io.read_model("../models/ardeche-train.hdf5")
df_train = model_to_df(model_train)

model_test = smash.io.read_model("../models/ardeche-test.hdf5")
df_test = model_to_df(model_test)

In [ ]:
df = df_train[df_train.code==df_train.code[0]]
df = df.drop(["code", "timestep"], axis=1)
ax = pd.plotting.scatter_matrix(df, alpha=0.2, diagonal='kde')

In [ ]:
df = df_test[df_test.code==df_test.code[0]]
df = df.drop(["code", "timestep"], axis=1)
ax = pd.plotting.scatter_matrix(df, alpha=0.2, diagonal='kde')

In [ ]:
df_train.boxplot(column="bias", by="code")

In [ ]:
df_test.boxplot(column="bias", by="code")

In [ ]:
def feature_engineering(df: pd.DataFrame):

    df["year"] = df["timestep"] // np.max(df["timestep_in_year"])

    df["precipitation_cumsum"] = df.groupby(["code", "year"])["precipitation"].cumsum()
    df["pet_cumsum"] = df.groupby(["code", "year"])["pet"].cumsum()
    df["discharge_sim_cumsum"] = df.groupby(["code", "year"])["discharge_sim"].cumsum()

    df['sqrt_pet'] = np.sqrt(df['pet'])

    df = df.drop(["year", "pet"], axis=1)

    # df['precipitation_time'] = df['precipitation'] * df['timestep_in_year']
    # df['discharge_sim_time'] = df['discharge_sim'] * df['timestep_in_year']
    # df["sqrt_pet_time"] = df["sqrt_pet"] * df['timestep_in_year']

    # df['cross_prcp_dis'] = df['precipitation'] * df['discharge_sim']
    # df['cross_prcp_pet'] = df['precipitation'] * df["pet"]
    # df['cross_pet_dis'] = df['pet'] * df['discharge_sim']

    return df

In [ ]:
df_train = feature_engineering(df_train)
df_train.to_csv("data-P1.csv", index=False)

df_test = feature_engineering(df_test)
df_test.to_csv("data-P2.csv", index=False)